In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt   
import seaborn as sns            
from sklearn.preprocessing import OneHotEncoder, LabelEncoder
from collections import Counter
from catboost import CatBoostClassifier, Pool
from xgboost import XGBClassifier
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import accuracy_score, classification_report
import warnings
warnings.filterwarnings("ignore")

In [2]:
train = pd.read_csv("/kaggle/input/playground-series-s5e6/train.csv")
test  = pd.read_csv("/kaggle/input/playground-series-s5e6/test.csv")
submission = pd.read_csv('/kaggle/input/playground-series-s5e6/sample_submission.csv')
original = pd.read_csv("/kaggle/input/fertilizer-prediction/Fertilizer Prediction.csv")
print("Import Successful")

Import Successful


In [3]:
print(original.info())
print(original.isnull().sum()) #info of original set

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 9 columns):
 #   Column           Non-Null Count   Dtype 
---  ------           --------------   ----- 
 0   Temparature      100000 non-null  int64 
 1   Humidity         100000 non-null  int64 
 2   Moisture         100000 non-null  int64 
 3   Soil Type        100000 non-null  object
 4   Crop Type        100000 non-null  object
 5   Nitrogen         100000 non-null  int64 
 6   Potassium        100000 non-null  int64 
 7   Phosphorous      100000 non-null  int64 
 8   Fertilizer Name  100000 non-null  object
dtypes: int64(6), object(3)
memory usage: 6.9+ MB
None
Temparature        0
Humidity           0
Moisture           0
Soil Type          0
Crop Type          0
Nitrogen           0
Potassium          0
Phosphorous        0
Fertilizer Name    0
dtype: int64


In [4]:
print(train.info())
print(train.isnull().sum()) #info of train set

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 750000 entries, 0 to 749999
Data columns (total 10 columns):
 #   Column           Non-Null Count   Dtype 
---  ------           --------------   ----- 
 0   id               750000 non-null  int64 
 1   Temparature      750000 non-null  int64 
 2   Humidity         750000 non-null  int64 
 3   Moisture         750000 non-null  int64 
 4   Soil Type        750000 non-null  object
 5   Crop Type        750000 non-null  object
 6   Nitrogen         750000 non-null  int64 
 7   Potassium        750000 non-null  int64 
 8   Phosphorous      750000 non-null  int64 
 9   Fertilizer Name  750000 non-null  object
dtypes: int64(7), object(3)
memory usage: 57.2+ MB
None
id                 0
Temparature        0
Humidity           0
Moisture           0
Soil Type          0
Crop Type          0
Nitrogen           0
Potassium          0
Phosphorous        0
Fertilizer Name    0
dtype: int64


In [5]:
print(test.info())
print(test.isnull().sum()) #info of test set

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 250000 entries, 0 to 249999
Data columns (total 9 columns):
 #   Column       Non-Null Count   Dtype 
---  ------       --------------   ----- 
 0   id           250000 non-null  int64 
 1   Temparature  250000 non-null  int64 
 2   Humidity     250000 non-null  int64 
 3   Moisture     250000 non-null  int64 
 4   Soil Type    250000 non-null  object
 5   Crop Type    250000 non-null  object
 6   Nitrogen     250000 non-null  int64 
 7   Potassium    250000 non-null  int64 
 8   Phosphorous  250000 non-null  int64 
dtypes: int64(7), object(2)
memory usage: 17.2+ MB
None
id             0
Temparature    0
Humidity       0
Moisture       0
Soil Type      0
Crop Type      0
Nitrogen       0
Potassium      0
Phosphorous    0
dtype: int64


In [6]:
print(submission.info())
print(submission.isnull().sum()) #info of sample submission set

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 250000 entries, 0 to 249999
Data columns (total 2 columns):
 #   Column           Non-Null Count   Dtype 
---  ------           --------------   ----- 
 0   id               250000 non-null  int64 
 1   Fertilizer Name  250000 non-null  object
dtypes: int64(1), object(1)
memory usage: 3.8+ MB
None
id                 0
Fertilizer Name    0
dtype: int64


In [7]:
train["const"] = 1  
test["const"] = 1
original["const"] = 1

In [8]:
original_copy = original.copy()
for k in range(6):
    original = pd.concat([original,original_copy],axis=0)

In [9]:
def feature_eng(df):
    numerical_features = [col for col in df.select_dtypes(include=['int64', 'float64']).columns 
                      if col != 'id']
    for col in numerical_features:
        df[f'{col}_Binned'] = df[col].astype(str).astype('category')
    return df

train = feature_eng(train)
test = feature_eng(test)
original = feature_eng(original)

In [10]:
cat_cols = [col for col in train.select_dtypes(include=['object', 'category']).columns 
            if col != "Fertilizer Name"]

for i in cat_cols:
    label_enc = LabelEncoder()
    train[i] = label_enc.fit_transform(train[i])
    original[i] = label_enc.fit_transform(original[i])
    test[i] = label_enc.transform(test[i])

fer_label_enc = LabelEncoder()
train["Fertilizer Name"] = fer_label_enc.fit_transform(train["Fertilizer Name"])
original["Fertilizer Name"] = fer_label_enc.fit_transform(original["Fertilizer Name"])
for col in cat_cols:
    train[col] = train[col].astype("category")
    test[col] = test[col].astype("category")
    original[col] = original[col].astype("category")

X = train.drop(columns=["id", "Fertilizer Name"])
y = train["Fertilizer Name"]
X_test = test.drop(columns=["id"])

X_original = original.drop(columns=["Fertilizer Name"])
y_original = original["Fertilizer Name"]

In [11]:
params = {
        'objective': 'multi:softprob',  
        'num_class': len(np.unique(y)), 
        'max_depth': 7,
        'learning_rate': 0.03,
        'subsample': 0.8,
        'max_bin': 128,
        'colsample_bytree': 0.4, 
        'colsample_bylevel': 1,  
        'colsample_bynode': 1,  
        'tree_method': 'hist',  
        'random_state': 42,
        'eval_metric': 'mlogloss',
        'device': "cuda",
        'enable_categorical':True,
        'n_estimators':5000,
        'early_stopping_rounds':50,
    }

In [12]:
def mapk(actual, predicted, k=3):
    def apk(a, p, k):
        p = p[:k]
        score = 0.0
        hits = 0
        seen = set()
        for i, pred in enumerate(p):
            if pred in a and pred not in seen:
                hits += 1
                score += hits / (i + 1.0)
                seen.add(pred)
        return score / min(len(a), k)
    return np.mean([apk(a, p, k) for a, p in zip(actual, predicted)])

FOLDS = 10
skf = StratifiedKFold(n_splits=FOLDS, shuffle=True, random_state=42)

oof = np.zeros(shape = (len(train) ,y.nunique()))
pred_prob = np.zeros(shape = (len(test),y.nunique()))

xgb_model = XGBClassifier(**params)

map3_scores = []

for i, (train_idx, valid_idx) in enumerate(skf.split(X,y)):
    print('#' * 15, i+1, '#' *15)
    x_train, x_valid = X.iloc[train_idx],X.iloc[valid_idx]
    y_train, y_valid = y.iloc[train_idx],y.iloc[valid_idx]

    x_train = pd.concat([x_train,X_original], axis=0, ignore_index=True)
    y_train = pd.concat([y_train,y_original], axis=0, ignore_index=True)
    
    xgb_model.fit(
        x_train, 
        y_train, 
        eval_set=[(x_train, y_train), (x_valid, y_valid)], 
        verbose=100,
    )
    oof[valid_idx] = xgb_model.predict_proba(x_valid)
    pred_prob += xgb_model.predict_proba(X_test)  
    top_3_preds = np.argsort(oof[valid_idx], axis=1)[:, -3:][:, ::-1]  
    actual = [[label] for label in y_valid]
    map3_score = mapk(actual, top_3_preds)
    map3_scores.append(map3_score)  # Store the score
    print(f"✅ FOLD {i+1}: MAP@3 Score: {map3_score:.5f}")

avg_map3 = np.mean(map3_scores)
print(f"\n🎯 Average MAP@3 Score across all folds: {avg_map3:.5f}")

############### 1 ###############
[0]	validation_0-mlogloss:1.94540	validation_1-mlogloss:1.94561
[100]	validation_0-mlogloss:1.90741	validation_1-mlogloss:1.92787
[200]	validation_0-mlogloss:1.87725	validation_1-mlogloss:1.91869
[300]	validation_0-mlogloss:1.84965	validation_1-mlogloss:1.91225
[400]	validation_0-mlogloss:1.82310	validation_1-mlogloss:1.90698
[500]	validation_0-mlogloss:1.79827	validation_1-mlogloss:1.90257
[600]	validation_0-mlogloss:1.77427	validation_1-mlogloss:1.89880
[700]	validation_0-mlogloss:1.75115	validation_1-mlogloss:1.89561
[800]	validation_0-mlogloss:1.72881	validation_1-mlogloss:1.89287
[900]	validation_0-mlogloss:1.70779	validation_1-mlogloss:1.89041
[1000]	validation_0-mlogloss:1.68712	validation_1-mlogloss:1.88829
[1100]	validation_0-mlogloss:1.66771	validation_1-mlogloss:1.88668
[1200]	validation_0-mlogloss:1.64888	validation_1-mlogloss:1.88540
[1300]	validation_0-mlogloss:1.63070	validation_1-mlogloss:1.88420
[1400]	validation_0-mlogloss:1.61304	val

In [13]:
top_3_preds = np.argsort(pred_prob, axis=1)[:, -3:][:, ::-1]
top_3_labels = fer_label_enc.inverse_transform(top_3_preds.ravel()).reshape(top_3_preds.shape)
df_sub = pd.read_csv("/kaggle/input/playground-series-s5e6/sample_submission.csv")
submission = pd.DataFrame({
    'id': df_sub['id'],
    'Fertilizer Name': [' '.join(row) for row in top_3_labels]
})
submission.to_csv('submission.csv', index=False)
print("✅ Submission file saved as 'submission.csv'")

✅ Submission file saved as 'submission.csv'
